In [12]:
# All imports in necessary for the function to work
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep, time
from PIL import Image 
import pathlib
import os
sleep(1)

In [10]:
def locationNameToSatImage(place_name,filename='', zoom=2):
    "zoom levels 1 - 3, you may leave out the filename to choose default"
    start = time()
    if not filename:
        filename=place_name
    if ".jpg" not in filename or '.png' not in filename:
        filename = filename.strip()+'.png'
    
    zoom = 4 if zoom>4 else zoom
    zoom = 1 if zoom<1 else zoom
    zoom = int((zoom/4)*16);
    
    #Helper funtions
    #===========================================
    def clickBtn(btnObj, clicks):
        """this is a helper function to click on an element a given number of times"""
        for i in range(clicks):
            btnObj.click()
            sleep(.5)
    #===========================================
   
    
    driver = webdriver.Chrome() # creating a chrome instance
    driver.maximize_window() # maximize the window
    #wait = WebDriverWait(driver, 10)
    url = 'https://earthexplorer.usgs.gov/'
    driver.get(url) # open the url
    #wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'googleAddress')))
    searchBox = driver.find_element_by_name("googleAddress") # find the searchbox to type in
    searchBox.send_keys(place_name) # type the location we'er looking for
    driver.find_element_by_id("geoShow").click() # submit the search term for query
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'address')))
    
    """ 
        the site returns the most suitable match for the search term. together with its
        longitude and latitude. Assuming it is what we are looking for, we click on it, 
        to retrieve a remote image for the area using the cordinates returned. 
    """
    # extracting the details returned
    lat = driver.find_element_by_class_name('lat').text
    long = driver.find_element_by_class_name('lng').text
    address = driver.find_element_by_class_name('address')
    #sleep(5)
    address.click() # after extracting the details we click on it
    
    '''
    Most images will come with the a view of the entire continent
    This means we would have to zoom in on the area we want to a visible raduis   
    '''
    
    zoomBtn = driver.find_element_by_xpath("//button[@title='Zoom in']") # locate the zoom btn
    
    
    clickBtn(zoomBtn,zoom) # calling on our helper fxn to zoom 
    sleep(5)
    # with a visible image, we need to locate the iframe cotianing the actual satellite iamge
    map = driver.find_element_by_tag_name('iframe')
    
    location = map.location # find its location on the web page [dict]
    size = map.size # and its size [dict]
    " the zoom out button (-) will help us find where the window ends and we can crop up to it"
    zoomoutBtn = driver.find_element_by_xpath("//button[@title='Zoom out']") # locate the zoom btn
    
    # get buttons cordinates
    btnLocation = zoomoutBtn.location
    cropendX = btnLocation['x']
    cropendY = btnLocation['y']
    
    # extract important info about the geometry of the image
    x = location['x']
    y = location['y']
    w = size['width']
    h = size['height']
    
    # construct its sqaure size
    left = int(x)
    right = int(x + w)
    top = int(y)
    bottom = int(y + cropendY)
    
    driver.save_screenshot(filename) # take  screenshot of the entire page
    driver.quit() # close the browser
    
    """
        Now we have an image of the location in a screenshot 
        but the shot also contains all other browser elements
        we need to crop it to the actual satellite view 
        This done using geometrica params we extracted earlier
    """
    # use the PIL libray to open the image, crop and save it
    im = Image.open(filename)
    im = im.crop((left, top, right, bottom))
    im.save(filename)
    
    #     end = time()
    #     dur = end - start
    #     print(dur)
    return filename

In [13]:
locationNameToSatImage('suma ahenko, ghana', zoom=4)

44.97882127761841


'suma ahenko, ghana.png'

In [107]:
arr = [2, 2, 1, 1]
def smallestMode(arr):
    """a solution for the hackerank challenge, Migratory birds"""
    s = set(arr)
    count = 0
    val = 0

    for i in s:
        c = arr.count(i)
        if c>count:
            count = c
            val = i

    return val

smallestMode(arr)

# dic = {i:arr.count(i) for i in arr}
# h = max(dic.values())
# k = list(dic.keys())
# v =  list(dic.values())

# a = k[v.index(h)]
# dic



1

In [ ]:
# load the image into pandas
import pandas as pd
from matplotlib import pyplot as plt
W = {'img':["./cropped.jpg"]}
df = pd.DataFrame(W)

im = plt.imread('cropped.png')

plt.imshow(im)
plt.show()
